# Her skal vi undersøke dataen hentet og bruke Pandas/SQL for å forstå strukturen

For å hente filepath, må vi manuelt lage path, slik at alle kan åpne filen selv om den er på forskjellige plasser for hver av oss:

In [2]:
import pandas as pd
from pandasql import sqldf
import os

# Henter file path

notebook_directory = os.getcwd()
root = os.path.abspath(os.path.join(notebook_directory, ".."))
file_path = os.path.join(root, "data", "Frost_Observations")



Dataen vi hentet er laget med mange dictionaries, som igjen inneholder flere dictionaries. For å kun få ut relevant data kan vi filtrere gjennom python for enkelte "entries" i forskjellige dictionaries. Deretter kan vi gjøre det om til en dataframe ved bruk av pandas og se hvordan dataen ser ut

In [3]:
import json

with open(file_path,"r") as fil: #leser filen og lagrer den som "data"
    data=json.load(fil)

# filtrerer dataen for bare det som er relevant til å analysere
brukbar_data = [
    {
        "elementId": obs["elementId"],
        "sourceId": entry["sourceId"],
        "referenceTime": entry["referenceTime"],
        "value": obs["value"],
        "unit": obs["unit"]
    }
    for entry in data["data"]
    for obs in entry["observations"] 
]

df=pd.DataFrame(brukbar_data) #Dataframe

pd.set_option('display.max_columns', None) #justerer output display
pd.set_option('display.width', 1000)
print(df.head())

                                           elementId   sourceId             referenceTime  value     unit
0                        mean(relative_humidity P1D)  SN18700:0  2020-07-01T00:00:00.000Z   66.0  percent
1  boolean_clear_sky_weather(cloud_area_fraction ...  SN18700:0  2020-07-01T00:00:00.000Z    0.0      sum
2                          mean(air_temperature P1D)  SN18700:0  2020-07-01T00:00:00.000Z   13.6     degC
3                          mean(air_temperature P1D)  SN18700:0  2020-07-01T00:00:00.000Z   14.6     degC
4                        mean(relative_humidity P1D)  SN18700:0  2020-07-02T00:00:00.000Z   63.0  percent


Da har vi fått lagd en dataframe med pandas som viser relevant data. Det ser litt uryddig ut og det passer bedre å ha hver observasjon som kolonner og så verdiene nedover, så vi kan bruke pd.pivot til til å gjøre det mer ryddig. 

Men det at vi har 2 verdier for mean(air_temperature P1D) gjør at dette ikke fungerer. Foreløpig kan vi kombinere det ved å ta gjennomsnittsverdien av de 2 målte, slik at vi kan visualisere dataframen bedre. Vi kan bruke "groupby" funksjonen for å samle de 2 identiske "mean(air_temperature P1D)" og slå sammen verdiene med gjennomsnittet. 

In [10]:
rader=[]

# henter relevant data ved å iterere gjennom i forskjellige dictionaries
for entry in data["data"]:
    referenceTime = entry["referenceTime"]
    for obs in entry["observations"]:
         rader.append({
            "referenceTime": referenceTime,
            "elementId": obs["elementId"],
            "value": obs["value"],
        })
         
df=pd.DataFrame(rader) #dataframe

# slår sammen der det er 2 av mean(air_temperature P1D) og tar gjennomsnittet, 
# gjør pivot av dataframe, slik at kolonnene blir type observasjon

df_gruppering = df.groupby(["referenceTime", "elementId"], as_index=False)["value"].mean()
df2=df_gruppering.pivot(index="referenceTime",columns="elementId",values="value")
df2=df2.reset_index()

# lagrer df2 som json fil for videre bruk
df2.to_json('../data/df2_data.json',indent=4)

print(df2.head())

elementId             referenceTime  boolean_clear_sky_weather(cloud_area_fraction P1D)  mean(air_temperature P1D)  mean(relative_humidity P1D)
0          2020-07-01T00:00:00.000Z                                                0.0                       14.10                         66.0
1          2020-07-02T00:00:00.000Z                                                0.0                       13.75                         63.0
2          2020-07-03T00:00:00.000Z                                                0.0                       12.85                         67.0
3          2020-07-04T00:00:00.000Z                                                0.0                       13.20                         63.0
4          2020-07-05T00:00:00.000Z                                                0.0                       14.15                         83.0


Her ser vi dataen på en mer forståelig måte. Det er verdt å merke at "boolean_clear_sky_weather(cloud_area_fraction P1D)" er boolean verdier, så i datahåndteringen kan vi ta hensyn til det og omgjøre det fra float. 

## Bruker PANDAS SQL (sqldf) til å hente data fra DataFrame

Nå som vi har en fungerende DataFrame, kan vi hente diverse type data og spesifikk data ved bruk av Pandas SQL/sqldf: 

1) Gjennomsnittlig temperatur for Juli i årene tilgjengelig
2) Alle dager der 24 timers gjennomsnittstemperaturen er over 20 grader og der det ikke var skyete basert på "boolean_clear_sky_weather..."

In [43]:
pysqldf = lambda q: sqldf(q, globals())

query = """

    SELECT strftime('%Y',referenceTime) AS År,
    AVG("mean(air_temperature P1D)") AS avg_temperatur_juli
    FROM df2
    WHERE strftime('%m',referenceTime) = '07'
    GROUP BY År
"""
resultat=pysqldf(query)
print(resultat)

     År  avg_temperatur_juli
0  2020            14.993548
1  2021            19.930645
2  2022            17.524194


In [57]:
query2="""
    SELECT * FROM df2
    WHERE "mean(air_temperature P1D)">=20.0
    AND "boolean_clear_sky_weather(cloud_area_fraction P1D)" == 1.0
    ORDER BY "mean(air_temperature P1D)" DESC;
"""
resultat=pysqldf(query2)
print(resultat)

              referenceTime  boolean_clear_sky_weather(cloud_area_fraction P1D)  mean(air_temperature P1D)  mean(relative_humidity P1D)
0  2021-07-03T00:00:00.000Z                                                1.0                       22.75                         43.0
1  2021-07-16T00:00:00.000Z                                                1.0                       22.75                         54.0
2  2021-07-24T00:00:00.000Z                                                1.0                       21.20                         55.0
3  2021-07-23T00:00:00.000Z                                                1.0                       21.00                         52.0
